# Генератор данных и действий пользователя
Применяется только для тестирования системы.  
Используются предзагруженная таблица данных о фильмах.

In [18]:
import logging
import random
import time
from datetime import datetime

import requests
from fake_useragent import UserAgent
from faker import Faker
from functools import wraps

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [19]:
def backoff(
    start_sleep_time: float = 0.2,
    factor: int = 2,
    border_sleep_time: int = 5,
    max_count_sleep: int = 3,
):
    def wrapper(func):
        @wraps(func)
        def inner(*args, **kwargs):
            sleep_time = start_sleep_time
            sleep_iter = 0
            while True:
                try:
                    res = func(*args, **kwargs)
                    return res
                except Exception as ex:
                    logger.info(
                        f"Exception in {func.__qualname__}: \n{ex}. \n"
                        f"Start sleeping for {sleep_time} seconds ({sleep_iter} iter)"
                    )
                    time.sleep(sleep_time)

                    sleep_iter += 1
                    sleep_time *= factor

                    if sleep_time >= border_sleep_time:
                        sleep_time = border_sleep_time

                    if sleep_iter >= max_count_sleep:
                        raise ex

        return inner

    return wrapper

In [20]:
ALL_GENRES = [
    "Western",
    "Adventure",
    "Drama",
    "Romance",
    "Sport",
    "Talk-Show",
    "Action",
    "Thriller",
    "Comedy",
    "Family",
    "Music",
    "Crime",
    "Animation",
    "Sci-Fi",
    "Documentary",
    "Musical",
    "Short",
    "Fantasy",
    "War",
    "Biography",
    "Mystery",
    "Reality-TV",
    "History",
    "News",
    "Horror",
    "Game-Show",
]

In [21]:
def generate_user(genres_count: int) -> dict:
    fake = Faker()
    profile = fake.profile()
    return {
        "login": profile["username"],
        "email": profile["mail"],
        "password": fake.password(length=10),
        "first_name": profile["name"].split()[0],
        "last_name": profile["name"].split()[1],
        "country": fake.country(),
        "city": fake.city(),
        "age": str(datetime.combine(profile["birthdate"], datetime.min.time())),
        "genres": random.sample(ALL_GENRES, k=genres_count),
    }


def get_short_profile_dict_by_fields(profile: dict, fields: list[str]) -> dict:
    return {key: value for key, value in profile.items() if key in fields}

In [22]:
generate_user(random.randint(0, len(ALL_GENRES) / 2))

{'login': 'toddadams',
 'email': 'samuel54@yahoo.com',
 'password': 'sX8Jv%(r1@',
 'first_name': 'Roberto',
 'last_name': 'White',
 'country': 'Seychelles',
 'city': 'North Jessica',
 'age': '2014-05-27 00:00:00',
 'genres': ['Thriller', 'Sport', 'Short']}

In [6]:
class GeneratorException(Exception):
    def __init__(self, msg):
        self.msg = msg

In [7]:
user_agent = UserAgent()


def create_headers(token: str | None = None):
    headers = {
        "Content-type": "application/json",
        "Accept": "application/json",
        "User-Agent": user_agent.random,
        "X-Request-Id": "request_id_00000000",
    }
    if token:
        headers["Authorization"] = f"Bearer {token}"
    return headers

In [8]:
@backoff()
def signup(headers: dict, user_data: dict):
    response = requests.post(
        "http://localhost:8001/api/v1/signup",
        json=get_short_profile_dict_by_fields(user_data, ["login", "email", "password"]),
        headers=headers,
    )
    response.raise_for_status()
    if not response.json()["success"]:
        raise GeneratorException(f"Ошибка создания пользователя {user_data}")

    logging.info(f"Создан пользователь {user_data['login']}")


@backoff()
def signin(headers: dict, user_data: dict) -> str:
    response = requests.post(
        "http://localhost:8001/api/v1/signin",
        json=get_short_profile_dict_by_fields(user_data, ["login", "password"]),
        headers=headers,
    )
    response.raise_for_status()
    if not response.json().get("data", {}).get("access_token"):
        raise GeneratorException(f"Ошибка получения токена для пользователя {user_data}")

    logging.info(
        f"Выполнен вход в учетную запись для пользователя {user_data['login']}, токен: {response.json()['data']['access_token']}"
    )
    return response.json()["data"]["access_token"]


def signup_signin(user_data: dict) -> str:
    """
    Функция создает аккаунт в сервисе auth, логинится и возвращает access токен
    """
    headers = create_headers()
    try:
        signup(headers, user_data)
    except Exception:
        logging.exception("Ошибка создания пользователя")

    try:
        token = signin(headers, user_data)
    except Exception:
        raise GeneratorException(f"Ошибка получения токена для пользователя")

    return token

In [9]:
# user_data = generate_user(random.randint(1, 5))
# token = signup_signin(user_data)
# create_profile_with_genres(user_data, token)

In [10]:
@backoff()
def create_profile(headers: dict, user_data: dict):
    response = requests.post(
        "http://localhost:8002/api/v1/user_profile/register_user_profile",
        json=get_short_profile_dict_by_fields(
            user_data, ["first_name", "last_name", "country", "city", "age"]
        ),
        headers=headers,
    )
    response.raise_for_status()


@backoff()
def create_profile_genre(headers: dict, user_data: dict, genre: str):
    response = requests.post(
        "http://localhost:8002/api/v1/user_profile/register_user_genre",
        json={"name": genre},
        headers=headers,
    )
    response.raise_for_status()


def create_profile_with_genres(user_data: dict, token: str):
    """
    Функция создает профиль пользователя в сервисе user_profile
    """
    headers = create_headers(token)
    try:
        create_profile(headers, user_data)
    except Exception:
        logging.exception(f"Ошибка создания профиля пользователя {user_data}")

    for genre in user_data["genres"]:
        try:
            create_profile_genre(headers, user_data, genre)
        except Exception:
            logging.exception(f"Ошибка добавления жанра к профилю пользователя {user_data}")

    logging.info(
        f"Создан профиль для пользователя {user_data['login']} с жанрами: {user_data['genres']}"
    )

In [11]:
def get_all_movies_ids_list() -> list[str]:
    """
    Функция возвращает список идентификаторов всех фильмов
    """
    headers = create_headers()
    movies_ids = []
    page = 1
    try:
        while page < 1000:
            response = requests.get(
                "http://localhost:8003/api/v1/movies/films/",
                params=dict(page_number=page, page_size=50),
                headers=headers,
            )
            response.raise_for_status()
            response = [movie["id"] for movie in response.json()]
            if len(response) == 0:
                break
            movies_ids.extend(response)
            page += 1
    except Exception:
        logging.exception(f"Ошибка получения списка фильмов")
    return movies_ids

In [12]:
@backoff()
def register_rating(headers: dict, movie_id: str):
    response = requests.put(
        "http://localhost:8004/api/v1/ugc/register_rating",
        json={
            "movie_id": movie_id,
            "rating": random.randint(1, 10),
            "created": str(datetime.now()),
        },
        headers=headers,
    )
    response.raise_for_status()


@backoff()
def register_review(headers: dict, movie_id: str):
    response = requests.put(
        "http://localhost:8004/api/v1/ugc/register_review",
        json={
            "movie_id": movie_id,
            "summary": "summary",
            "description": "description",
            # score по вероятности 0.5
            "score": ("positive", "negative")[bool(random.getrandbits(1))],
            "created": str(datetime.now()),
        },
        headers=headers,
    )
    response.raise_for_status()


def create_ratings_and_reviews(token: str, movies_ids: list[str], review_probability: float):
    """
    Функция создет ugc для фильмов: рейтинги, отзывы
    """
    headers = create_headers(token)
    reviews_count = 0

    for movie_id in movies_ids:
        try:
            register_rating(headers, movie_id)
        except Exception:
            logging.exception(f"Ошибка регистрации рейтинга")

        if random.random() < review_probability:
            try:
                register_review(headers, movie_id)
            except Exception:
                logging.exception(f"Ошибка регистрации отзыва")

            reviews_count += 1

    logging.info(
        f"Создано {len(movies_ids)} оценок и {reviews_count} отзывов для фильмов по токену {token}"
    )

In [13]:
@backoff()
def create_bookmark(headers: dict, token: str, movie_id: str):
    response = requests.put(
        "http://localhost:8004/api/v1/ugc/register_bookmark",
        json={
            "movie_id": movie_id,
            "action": "add",
            "created": str(datetime.now()),
        },
        headers=headers,
    )
    response.raise_for_status()


def create_bookmarks(token: str, movies_ids: list[str]):
    """
    Функция создет ugc для фильмов: закладки
    """
    headers = create_headers(token)
    for movie_id in movies_ids:
        try:
            create_bookmark(headers, token, movie_id)
        except Exception:
            logging.exception(f"Ошибка при создании закладки")

    logging.info(f"Создано {len(movies_ids)} закладок фильмов по токену {token}")

In [14]:
@backoff()
def create_movie_progress(headers: dict, token: str, movie_id: str):
    response = requests.put(
        "http://localhost:8004/api/v1/ugc/register_movie_progress",
        json={
            "movie_id": movie_id,
            "progress": random.random() * 120,
            "created": str(datetime.now()),
        },
        headers=headers,
    )
    response.raise_for_status()


def create_movies_progress(token: str, movies_ids: list[str]):
    """
    Функция создет ugc для фильмов: продолжительность просмотра фильма
    """
    headers = create_headers(token)
    for movie_id in movies_ids:
        try:
            create_movie_progress(headers, token, movie_id)
        except Exception:
            logging.exception(f"Ошибка при создании просмотра")

    logging.info(f"Созданы просмотры по {len(movies_ids)} фильмам по токену {token}")

In [15]:
N_USERS = 1000

all_movies_ids = get_all_movies_ids_list()

for i in range(N_USERS):
    logging.info(f"Пользователь {i}")
    user_data = generate_user(random.randint(1, 5))
    try:
        token = signup_signin(user_data)
    except GeneratorException:
        continue

    create_profile_with_genres(user_data, token)

    watched_movies = random.sample(all_movies_ids, k=random.randint(20, 100))
    create_movies_progress(token, watched_movies)

    rated_movies = random.sample(watched_movies, k=random.randint(20, len(watched_movies)))
    create_ratings_and_reviews(token, rated_movies, review_probability=0.2)

    movies_in_bookmarks = list(
        set(random.sample(watched_movies, k=random.randint(10, int(len(watched_movies) / 2))))
        | set(random.sample(all_movies_ids, k=random.randint(0, 40)))
    )
    create_bookmarks(token, movies_in_bookmarks)

INFO:root:Пользователь 0
INFO:root:Создан пользователь victoria22
INFO:root:Выполнен вход в учетную запись для пользователя victoria22, токен: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJmcmVzaCI6ZmFsc2UsImlhdCI6MTY5NTQ5NjQ1NiwianRpIjoiZDQ1ZDVmYWItZmUwNC00ZTU0LWJiOGMtYjE2ZGJjYTI4ZDQ4IiwidHlwZSI6ImFjY2VzcyIsInN1YiI6eyJpZCI6ImI1Yjg1ZGU2LWYxNjEtNDhjZC1hYWE2LWM3M2UzZDQzODU0MSIsImRldmljZV9pbmZvIjoiTW96aWxsYS81LjAgKFdpbmRvd3MgTlQgMTAuMDsgV2luNjQ7IHg2NCkgQXBwbGVXZWJLaXQvNTM3LjM2IChLSFRNTCwgbGlrZSBHZWNrbykgQ2hyb21lLzExNC4wLjAuMCBTYWZhcmkvNTM3LjM2IEVkZy8xMTQuMC4xODIzLjg2IiwiaXNfYWN0aXZlIjoiVHJ1ZSIsImlzX3ZlcmlmaWVkIjoiRmFsc2UiLCJpc19hZG1pbiI6IkZhbHNlIiwicm9sZXMiOlsiZGVmYXVsdCJdfSwibmJmIjoxNjk1NDk2NDU2LCJjc3JmIjoiN2JjYzI0MzQtMjYzOC00ZWQzLThmNmMtM2RiODQ1NTFkM2RmIiwiZXhwIjoxNjk1NTAwMDU2fQ.qtWXTvvLF1EMtEarlXlJyk97CqgP-CwVOHOxMopxE6E
INFO:root:Создан профиль для пользователя victoria22 с жанрами: ['Animation', 'Fantasy', 'History', 'Music', 'Talk-Show']
INFO:root:Созданы просмотры по 26 фильмам по токену

10


### Получить дамп заполненных баз:  

docker-compose exec -T db-auth pg_dump auth_db -U app > auth_database.sql  

docker-compose exec -T db-user-profile pg_dump user_profile_database -U admin > user_profile_database.sql  

docker-compose exec -T mongo mongodump --db ugc --gzip --archive=ugc_database.gz  
docker cp mongo:/ugc_database.gz .  

docker-compose exec redis-user-data redis-cli save  
docker cp redis-user-data:/data/dump.rdb ./redis_user_database.rdb  

Необходимо перенести их в директорию db_dumps